In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from math import exp, log, cos, cosh, sqrt, sinh, pi
from time import process_time, time
from random import uniform

In [ ]:
from scipy.integrate import quad

In [ ]:
@jit
def f(t_rn, h, T, nj):
    return exp(-h*cos(2*pi*t_rn/T)*nj)

@jit 
def g(t_rn, J, h, T, ni, nj):
    return exp(-J*ni*nj-h*np.cos(2*pi*t_rn/T)*nj)

In [ ]:
J = 4
h = 0.2
T = 10
ni = -1
nj = 1

In [ ]:
ts = np.linspace(0, T, num = 2001)
signals_1 = np.zeros(2001)
signals_2 = np.zeros(2001)
signals_3 = np.zeros(2001)
signals_4 = np.zeros(2001)
integrated_signals= np.zeros(2001)

for i in range(2001):
    signals_1[i] = g(ts[i], J, h, T, 1, 1)
    signals_2[i] = g(ts[i], J, h, T, 1, -1)
    signals_3[i] = g(ts[i], J, h, T, -1, 1)
    signals_4[i] = g(ts[i], J, h, T, -1, -1)
    
    #integrated_signals[i] = quad(g, 0, ts[i], args=(J, h, T, ni, nj))[0]
    
plt.scatter(ts, signals_1)
plt.scatter(ts, signals_2)
plt.scatter(ts, signals_3)
plt.scatter(ts, signals_4)
plt.legend(['1, 1', '1, -1', '-1, 1', '-1, -1'])
#plt.scatter(ts, integrated_signals)

In [ ]:
ts = np.linspace(0, T, num = 1001)
signals_1 = np.zeros(1001)
signals_2 = np.zeros(1001)
signals_3 = np.zeros(1001)
signals_4 = np.zeros(1001)
integrated_signals= np.zeros(1001)

for i in range(1001):
    signals_1[i] = g(ts[i], J, -0.2, T, 1, 1)
    signals_2[i] = g(ts[i], J, -0.2, T, 1, -1)
    signals_3[i] = g(ts[i], J, -0.2, T, -1, 1)
    signals_4[i] = g(ts[i], J, -0.2, T, -1, -1)
    
    #integrated_signals[i] = quad(g, 0, ts[i], args=(J, h, T, ni, nj))[0]
    
plt.scatter(ts, signals_1)
plt.scatter(ts, signals_2)
plt.scatter(ts, signals_3)
plt.scatter(ts, signals_4)
plt.legend(['1, 1', '1, -1', '-1, 1', '-1, -1'])
#plt.scatter(ts, integrated_signals)

In [ ]:
int_1 = quad(f, 14.91, 15, args=(h, T, nj))[0] 
int_2 = quad(f, 14.91, 14.92, args=(h, T, nj))[0] 

print(int_2-int_1, np.abs(int_2-int_1)/int_2)

In [2]:
from tRSSA_1D_Ising_Chain import TD_1DChain

In [3]:
def last_n_nonzero(M, n):
    '''
    For a matrix M of m by N, 
    return a matrix of m by n by removing the zero at the end of each row and keep the last n element 
    '''
    m, N = M.shape
    New_M = np.zeros((m, n))
    for i in range(m):
        row_i = np.trim_zeros(M[i], 'b')
        New_M[i] = row_i[-n:]
    return New_M   

In [7]:
#initial_chain = np.random.choice([-1,1], size=10000)

                  
T = 100
J = 4
h = 0.2
dmu = 7
Periods = 500
Burn_in_Periods = 300

t_max = T*Periods
t_burn_in = T*Burn_in_Periods
prop_t_bounds = np.linspace(0, t_max, num=T*Periods*5+1)
num_t_save_intervals = 200
t_save_intervals = np.linspace(t_burn_in, t_max, num=num_t_save_intervals)

t_start = process_time() 
chain_length, patterns_save, addition_reaction_time_save = TD_1DChain(T, dmu, h, 
                                                                    t_max = t_max, 
                                                                    prop_t_bounds=prop_t_bounds,
                                                                    t_save_intervals = t_save_intervals,
                                                                    seed =None)
t_end = process_time() 
print(t_end-t_start)

print(chain_length)                         

3427292 1554484
0.9375
1873808


In [ ]:
print(addition_reaction_time_save.shape)

In [ ]:
print(addition_reaction_time_save)

In [ ]:
print(last_n_nonzero(addition_reaction_time_save, 100))
print(last_n_nonzero(patterns_save, 100))

In [17]:
from joblib import Parallel, delayed

In [8]:
def tSRRA_1D_Chain_Process(T, h, t_max, t_burn_in, num_t_save_intervals, J=4, dmu=7, last_n = 100, save='', save_n = 0):
    prop_t_bounds = np.linspace(0, t_max, num=t_max*10+1)
    t_save_intervals = np.linspace(t_burn_in, t_max, num=num_t_save_intervals)
    
    chain_length, patterns, addition_reaction_time = TD_1DChain(T, dmu, h, t_max = t_max,
                                                                        prop_t_bounds=prop_t_bounds,
                                                                        t_save_intervals = t_save_intervals)
        
    patterns_save = last_n_nonzero(patterns, last_n)
    addition_reaction_time_save = last_n_nonzero(addition_reaction_time, last_n)
    
    vals_to_save = {'chain_length':chain_length,
                        'patterns': patterns_save, 'addition_reaction_time': addition_reaction_time_save}
    save_str = 'data/' + save+ '_T='+str(T)+'_h='+str(h)+'_'+str(save_n)
    np.savez(save_str, **vals_to_save)

In [37]:
t_start = process_time()
Ts = np.array([10, 100])
hs = np.array([-0.2, 0.2])
for T in Ts:
    for h in hs:
        for i in range(10):
            tSRRA_1D_Chain_Process(T, h, t_max, t_burn_in, num_t_save_intervals, J=4, dmu=7, last_n = 100, save='NOV27', save_n = i)
t_end = process_time()
print(t_end - t_start)

2668302 1204314
2667511 1205903
2671695 1206780
2665797 1204873
2668207 1208013
2663383 1207277
2672280 1202923
2664436 1206082
2667618 1205716
2665863 1205413
3427865 1540681
3429236 1540547
3422665 1547594
3423085 1544700
3421329 1548813
3427932 1541166
3427250 1543803
3423215 1544923
3423008 1546346
3422231 1547890
2670478 1205553
2671646 1206193
2670564 1207447
2672848 1211506
2669104 1206843
2675244 1205212
2666921 1208109
2674851 1205813
2673566 1208558
2670758 1205177
3431767 1551504
3426741 1553254
3430830 1545368
3432702 1547985
3425925 1549916
3431185 1547658
3431646 1546213
3425175 1551501
3434222 1543321
3429693 1544627
43.90625


In [18]:
t_start = time()
Parallel(n_jobs=-1)(delayed(tSRRA_1D_Chain_Process) (T, h, t_max, t_burn_in, num_t_save_intervals, save='NOV27', save_n = i)
                   for i in range(10) for T in [1000])
t_end = time()
print(t_end - t_start)

10.796996593475342


In [35]:
def Run_tSRRA_1D_Chain_Process(Ts, hs, t_max, num_save, trials = 100,
                               t_burn_in = 30*1000, J=4, dmu=7, 
                               last_n = 100, 
                               n_jobs = 1, save=''):
    for T in range(len(Ts)):
        T_i = Ts[T]
        for h in range(len(hs)):
            h_i = hs[h]
            Parallel(n_jobs=n_jobs)(delayed(tSRRA_1D_Chain_Process)(T_i, h_i, t_max, t_burn_in, num_save, 
              J=J, dmu=dmu, last_n = last_n, save=save, save_n = i) for i in range(trials))  

In [36]:
Ts = np.array([10, 100])
hs = np.array([-0.2, 0.2])
t_max = 50*1000
num_save = 200

t_start = time()
Run_tSRRA_1D_Chain_Process(Ts, hs, t_max, num_save, trials = 25, save='DEC01', n_jobs=-1) 
t_end = time()
print(t_end - t_start)

35.26301670074463


In [ ]:
simulation_time = tSRRA_1D_Chain_Process(T, h, Periods, Burn_in_Periods, J=4, dmu=5.5, last_n = 100, save='NOV27')


In [ ]:
print(np.mean(simulation_time))

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.set_yscale('log')
methods = ['tRSSA', 'FRM']
speeds = [np.mean(simulation_time),25.7]
ax.bar(methods,speeds)
ax.set_ylabel('CPU time (seconds)', fontsize=16)
ax.set_xlabel('Simulation Methods', fontsize=16)
ax.set_title('CPU time for simulating 50 periods', fontsize=16)
fig.savefig('CPUtime_vs_methods.png', dpi=200)

In [ ]:
print(addition_reaction_time[0:20])

In [ ]:
#trim the 0s
chain = np.trim_zeros(chain, 'b')
Chain_Changes = np.trim_zeros(Chain_Changes, 'b')
addition_reaction_time = np.trim_zeros(addition_reaction_time, 'b')

In [ ]:
trials = 9000
for trial_i in range(trials):
    chain_length, chain, Chain_Changes, addition_reaction_time = TD_1DChain(T, dmu, h, 
                                                               Periods = Periods, t_intervals=ts)
    chain = np.trim_zeros(chain, 'b')
    Chain_Changes = np.trim_zeros(Chain_Changes, 'b')
    addition_reaction_time = np.trim_zeros(addition_reaction_time, 'b')
    
    print(chain_length)
    
    vals_to_save = {'chain_length':chain_length,
                    'chain': chain, 'Chain_Changes':Chain_Changes, 'addition_reaction_time': addition_reaction_time}
    save_str = 'data/T='+str(T)+'_h='+str(h)+'_'+str(trial_i+1000)
    np.savez(save_str, **vals_to_save)

In [ ]:
def bits_patterns(n):
    '''
    Generate all patterns of 1&-1's of size n
    '''
    ps = ['-1', '1']
    for i in range(n-1):
        ps_temp = []
        for i in range(len(ps)):
            ps_temp.append(ps[i]+'-1')
            ps_temp.append(ps[i]+'1')
        ps = ps_temp
    return ps  

In [ ]:
def bits_pattern_dict(n):
    two_to_n = 2**n
    n_bits_patterns = bits_patterns(n)
    #create a dictionary to store patterns index
    n_bits_patterns_dict = {}
    for i in range(two_to_n):
        n_bits_patterns_dict[n_bits_patterns[i]] = i
    return n_bits_patterns_dict

In [ ]:
n_trials = 10000
n_sample_interval = 1000
n_samples = int(n_trials/n_sample_interval)
n_sites = 10
num_patterns = 2**n_sites

In [ ]:
n_bits_ps_dict = bits_pattern_dict(n_sites)
#store the running frequencies
n_bits_ps_freq = np.zeros( len(n_bits_ps_dict) ) 

In [ ]:
#save the distributions for n_samples
n_bits_ps_dist = []
#analyze distributions
for i in range(n_trials):
    data = np.load('data/T='+str(T)+'_h='+str(h)+'_'+str(i)+'.npz')
    last_n_sites = "".join(map(str, data['chain'][-n_sites:].astype(int))) 
    n_bits_ps_freq[n_bits_ps_dict[last_n_sites]] += 1
    if i > 0 and (i+1)%n_sample_interval==0:
        n_bits_ps_dist.append(n_bits_ps_freq/i)

n_bits_ps_dist = np.array(n_bits_ps_dist)
    

In [ ]:
x_array = np.arange(num_patterns)
for i in range(n_samples):
    plt.scatter(x_array, n_bits_ps_dist[i], s=1)
    plt.legend([str(n_sample_interval*(i+1))])
    plt.ylim(0, 0.0025)
    save_str = '10sites_dist_samples='+ str(n_sample_interval*(i+1)) +'.png'
    plt.ylabel('Probability', size=16)
    plt.xlabel('Pattern Labels', size=16)
    plt.title('10 Sites Pattern Distributions', size=16)
    plt.savefig(save_str, dpi=200)
    plt.show()
#plt.legend(['1000', '2000', '3000', '4000', '5000', '6000', '7000', '8000', '9000'])

In [ ]:
from scipy.special import kl_div

In [ ]:
kl_div_to_final = np.zeros(n_samples)
for i in range(n_samples-1):
    kl_div_to_final[i] = np.sum(kl_div(n_bits_ps_dist[i], n_bits_ps_dist[n_samples-1]))

In [ ]:
n_samples_array = (np.arange(n_samples)+1)*n_sample_interval
plt.scatter(n_samples_array[:-1], kl_div_to_final[:-1])
plt.title('KL Divergence to Final Distributions of 10 sites')
plt.xlabel('Number of Samples', size=16)
plt.savefig('10sites_KLDIV_vs_samples.png', dpi=200)

In [ ]:
print( kl_div_to_final)